# Mole Classifier using InceptionV3

In [3]:
# Step 1: Importing Essential Libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
folder_benign_train = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/train/No Cancer'
folder_malignant_train = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/train/Cancer'

folder_benign_test = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/test/No Cancer'
folder_malignant_test = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/test/Cancer'  # Change this to your dataset path
CATEGORIES = ['benign', 'malignant']
IMG_SIZE = 224

In [8]:
from tqdm import tqdm
import os
# ... other imports

folder_benign_train = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/train/No Cancer'  # Update with your path

# Check if the directory exists
if not os.path.exists(folder_benign_train):
    raise FileNotFoundError(f"Error: Directory not found: {folder_benign_train}")

data = []
for img in tqdm(os.listdir(folder_benign_train)):
    img_path = os.path.join(folder_benign_train, img)
    img_array = cv2.imread(img_path)
    # ... (image preprocessing steps: resize, etc.) ...
    data.append([img_array, 0])  # 0 for benign

# Process malignant training data
for img in tqdm(os.listdir(folder_malignant_train)):
    img_path = os.path.join(folder_malignant_train, img)
    img_array = cv2.imread(img_path)
    # ... (image preprocessing steps: resize, etc.) ...
    data.append([img_array, 1])  # 1 for malignant

# Process benign test data
for img in tqdm(os.listdir(folder_benign_test)):
    img_path = os.path.join(folder_benign_test, img)
    img_array = cv2.imread(img_path)
    # ... (image preprocessing steps: resize, etc.) ...
    data.append([img_array, 0])  # 0 for benign

# Process malignant test data
for img in tqdm(os.listdir(folder_malignant_test)):
    img_path = os.path.join(folder_malignant_test, img)
    img_array = cv2.imread(img_path)
    # ... (image preprocessing steps: resize, etc.) ...
    data.append([img_array, 1])  # 1 for malignant

100%|██████████| 100/100 [00:05<00:00, 19.91it/s]


In [10]:
# Step 3: Split Features and Labels
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = to_categorical(y, num_classes=2)

In [11]:
# Step 4: Normalize the Data
X = X / 255.0

In [15]:
from tqdm import tqdm
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# ... (other imports) ...

folder_benign_train = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/train/No Cancer'
folder_malignant_train = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/train/Cancer'
folder_benign_test = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/test/No Cancer'
folder_malignant_test = '/content/drive/MyDrive/Skin Cancer Detection/Cancer Data/test/Cancer'
CATEGORIES = ['benign', 'malignant']
IMG_SIZE = 224

data = []

def load_and_preprocess_images(folder_path, label):
    """Loads images from a folder, resizes them, and appends them to the data list."""
    for img in tqdm(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, img)
        # Use cv2.IMREAD_COLOR to ensure images are loaded in color
        img_array = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img_array is not None:  # Check if image was loaded successfully
            img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append([img_array, label])
        else:
            print(f"Warning: Could not load image: {img_path}")


# Load and preprocess images from all folders
load_and_preprocess_images(folder_benign_train, 0)  # 0 for benign
load_and_preprocess_images(folder_malignant_train, 1)  # 1 for malignant
load_and_preprocess_images(folder_benign_test, 0)  # 0 for benign
load_and_preprocess_images(folder_malignant_test, 1)  # 1 for malignant

# Split Features and Labels
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = to_categorical(y, num_classes=2)

# Normalize the Data
X = X / 255.0

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

100%|██████████| 100/100 [00:01<00:00, 65.56it/s]


In [16]:
# Step 5: Train-Test Split
# ... (other imports)

# Remove the data initialization and the for loops in this cell
# The data list should have been populated in cell 8
# ... (Remove lines 10-25 from the original code) ...

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Step 6: Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [18]:
# Step 7: Load InceptionV3 and Construct Model
base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 111, 111, 32)   │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 111, 111, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 109, 109, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 109, 109, 32)   │             96 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 109, 109, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 109, 109, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 109, 109, 64)   │            192 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 109, 109, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 54, 54, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 54, 54, 80)     │          5,120 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 54, 54, 80)     │            240 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 54, 54, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 52, 52, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 52, 52, 192)    │            576 │ conv2d_4[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activat

 Total params: 22,065,314 (84.17 MB)

 Trainable params: 262,530 (1.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [20]:
# Step 8: Train Model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    steps_per_epoch=len(X_train) // 32,
    epochs=10
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 718ms/step - accuracy: 0.9736 - loss: 0.0850 - val_accuracy: 0.9867 - val_loss: 0.0299
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 1.0000 - loss: 0.0143 - val_accuracy: 0.9867 - val_loss: 0.0313
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 714ms/step - accuracy: 0.9749 - loss: 0.0608 - val_accuracy: 1.0000 - val_loss: 0.0130
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.0228 - val_accuracy: 1.0000 - val_loss: 0.0115
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 491ms/step - accuracy: 0.9892 - loss: 0.0414 - val_accuracy: 1.0000 - val_loss: 0.0136
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0174 - val_accuracy: 1.0000 - val_loss: 0.0136
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 337ms/step - accuracy: 0.9927 - loss: 0.0117 - val_accuracy: 1.0000 - val_loss: 0.0119
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 1.0000 - val_loss: 0.0

In [21]:
# Step 9: Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Final Test Accuracy: {accuracy:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9894 - loss: 0.0137
Final Test Accuracy: 0.9867
